In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
Gemini_api_key = os.environ["GEMINI_API_KEY"]

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain import hub

In [4]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import AstraDB

In [5]:
GoogleGeminiEmbeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=Gemini_api_key )

In [6]:
loader = PyPDFLoader("./Brown_Churchill_Complex_Variables_and_Ap.pdf")
pages = loader.load_and_split()

In [7]:
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1200, chunk_overlap = 400)
splits = text_splitter.split_documents(pages)

In [21]:
vstore = AstraDB(
    embedding=GoogleGeminiEmbeddings,
    collection_name="Complex_variables",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [22]:
ids = vstore.add_documents(splits)

In [23]:
retriever = vstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.4, google_api_key=Gemini_api_key)

In [31]:
def pages_adjustments(pages):
    return "\n\n".join(page.page_content for page in pages)

In [32]:
context_chain = retriever | pages_adjustments

In [34]:
main_chain = (
    {"context" : context_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [39]:
main_chain.invoke("Give proof of this theorem If f(z) = 0 everywhere in a domain D, then f (z) must be constant throughout D")

'I apologize, but the proof of the theorem "If f(z) = 0 everywhere in a domain D, then f (z) must be constant throughout D" is not provided in the context you provided. Therefore, I cannot answer your question.'